In [1]:
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../src')))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../scripts')))
from data_cleaning.data_processor import DataPreprocessor
from sklearn.model_selection import train_test_split, GridSearchCV
# Simulated data for demonstration purposes
import numpy as np
from sales_prediction.tree_regessor_pipeline import TreeBasedRegressorPipeline




In [2]:
# Load the datasets
train_data = pd.read_csv('../data/train.csv')
test_data = pd.read_csv('../data/test.csv')
store_data = pd.read_csv('../data/store.csv')
sample_submission = pd.read_csv('../data/sample_submission.csv')


C:\Users\sefuwan.feysa\AppData\Local\Temp\ipykernel_19260\4150674729.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv('../data/train.csv')


In [3]:


preprocessor_train = DataPreprocessor(train_data)
processed_train_data = preprocessor_train.preprocess(store_df=store_data)

preprocessor_test = DataPreprocessor(test_data)
processed_test_data = preprocessor_test.preprocess(store_df=store_data)


d:\tutorial\10x\tasks\task_4\store-sales-forecast\scripts\data_cleaning\data_processor.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  self.df['CompetitionDistance'].fillna(self.df['CompetitionDistance'].median(), inplace=True)
d:\tutorial\10x\tasks\task_4\store-sales-forecast\scripts\data_cleaning\data_processor.py:26: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work

In [4]:
 # Print a preview of the processed data
print(processed_train_data.head())
print(processed_test_data.head())

      Store  DayOfWeek       Date     Sales  Customers      Open     Promo  \
0 -1.731640   0.501484 2015-07-31 -0.132683  -0.168269  0.452399  1.273237   
1 -1.728534   0.501484 2015-07-31  0.075373  -0.017540  0.452399  1.273237   
2 -1.725427   0.501484 2015-07-31  0.659800   0.404499  0.452399  1.273237   
3 -1.722321   0.501484 2015-07-31  2.135414   1.862258  0.452399  1.273237   
4 -1.719214   0.501484 2015-07-31 -0.247231  -0.159656  0.452399  1.273237   

   SchoolHoliday  CompetitionDistance  CompetitionOpenSinceMonth  ...  \
0       2.144211            -0.538742                   0.436247  ...   
1       2.144211            -0.629569                   1.155994  ...   
2       2.144211             1.129891                   1.515868  ...   
3       2.144211            -0.623082                   0.436247  ...   
4       2.144211             3.177404                  -1.363121  ...   

   StateHoliday_0  StateHoliday_a  StateHoliday_b  StateHoliday_c   Weekend  \
0            

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load your dataset
df = pd.read_csv('../data/train.csv')
# Convert 'Date' to datetime and extract useful features
# df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%Y')
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')

df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day

# Fill missing values for each column
df['Store'].fillna(df['Store'].mode()[0], inplace=True)
df['DayOfWeek'].fillna(df['DayOfWeek'].median(), inplace=True)
df.dropna(subset=['Date'], inplace=True)
df['Sales'].interpolate(method='linear', inplace=True)  # or drop rows with missing Sales
df['Customers'].fillna(df['Customers'].median(), inplace=True)
df['Open'].fillna(1, inplace=True)
df['Promo'].fillna(0, inplace=True)
df['StateHoliday'].fillna('0', inplace=True)
df['SchoolHoliday'].fillna(0, inplace=True)
# Drop the original 'Date' column
df = df.drop(columns=['Date'])

# Convert 'StateHoliday' to a one-hot encoded format (assuming it's categorical)
df['StateHoliday'] = df['StateHoliday'].astype(str)  # Ensure it's a string
df = pd.get_dummies(df, columns=['StateHoliday'], drop_first=True)

# Ensure no missing values (if any, fill them with 0 or an appropriate strategy)
df = df.fillna(0)

# Separate features (train_data) and labels (train_labels)
train_data = df.drop(columns=['Sales'])  # 'Sales' is the target
train_labels = df['Sales']

# Convert to NumPy arrays and ensure the data type is float32
train_data = np.array(train_data, dtype=np.float32)
train_labels = np.array(train_labels, dtype=np.float32)

# Split into training and test sets
train_data, test_data, train_labels, test_labels = train_test_split(train_data, train_labels, test_size=0.2, random_state=42)

C:\Users\sefuwan.feysa\AppData\Local\Temp\ipykernel_19260\49281902.py:5: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/train.csv')


ValueError: time data "2015-07-31" doesn't match format "%m/%d/%Y", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [8]:
# Example: TensorFlow model
from sales_prediction.sales_predictor import SalesPredictor


predictor = SalesPredictor(model_type='tensorflow', delta=1.0)
input_shape = (train_data.shape[1],)  # Replace with actual input shape
tf_model = predictor.build_tensorflow_model(input_shape)
history = predictor.train_tensorflow_model(train_data, train_labels)

# Example: Scikit-learn model
predictor = SalesPredictor(model_type='sklearn', delta=1.0)
predictor.build_sklearn_model()
predictor.train_sklearn_model(train_data, train_labels)
scores = predictor.evaluate_sklearn_model(test_data, test_labels)
print(f"Mean Huber Loss (cross-validated): {np.mean(scores)}")


d:\tutorial\10x\tasks\task_4\store-sales-forecast\.winvenv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).

: 

In [ ]:
X, y = np.random.rand(100, 5), np.random.rand(100)  # Replace with real data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the model pipeline with RandomForestRegressor
model_pipeline = TreeBasedRegressorPipeline(model_type='random_forest')

# Build the pipeline
model_pipeline.build_pipeline()

# Train the model
model_pipeline.train_model(X_train, y_train)

# Evaluate the model
metrics = model_pipeline.evaluate_model(X_test, y_test)
print(f"Evaluation Metrics: {metrics}")

# Hyperparameter tuning
param_grid = {
    'regressor__n_estimators': [50, 100, 200],
    'regressor__max_depth': [None, 10, 20]
}
best_params = model_pipeline.tune_hyperparameters(X_train, y_train, param_grid)
print(f"Best Hyperparameters: {best_params}")

# Predictions
predictions = model_pipeline.predict(X_test)
print(f"Predictions: {predictions[:5]}")
